- required packages
    - wave
    - numpy
- variables
    - feat_scp: the file path of target feature file that you want to calculate the mean and the standard variation
    - out_dir: the file path to which you want to write the mean and standardvariation of a dataset
- steps
    - read feature files
    - calculate the mean and the variance
    - note the mean and variance according to the dimension of a feature 

In [ ]:
if __name__ == '__main__':
    # the mean and std of fbank is going to be utilized for DNN-HMM
    # the mean and std of mfcc is going to be utilized for DP Mathinc and GMM-HMM
    
    feature_list = ['fbank', 'mfcc'] 

    for feature in feature_list:
        feat_scp = 'the file where list of metadata of feature files are saved' % (feature)
        out_dir = 'a file path to note the mean and std of feature dataset' % (feature)
        
        Sum = None
        Sum_Squared = None
        total_frames = 0
        
        with open(feat_scp, mode = 'r') as file_feat:
            # extract mean and std for each dataset(e.g. train/fbank, dev/fbank, test/fbank, train/mfcc, dev/mfcc, test/mfcc)
            for i, line in enumerate(file_feat):
                parts = line.split()
                utterance_id = parts[0]
                feat_path = parts[1]
                num_frames = int(parts[2])
                num_dims = int(parts[3])
                
                # read the feature values
                feature = np.fromfile(feat_path, dtype = np.float32)
                feature = feature.reshape((num_frames, num_dims))
                
                # initialize the value of calculation variable
                if i == 0:
                    Sum = np.zeros(num_dims, np.float32)
                    Sum_squared = np.zeros(num_dims, np.float32)
                    
                total_frames += num_frames
                Sum += np.sum(feature, axis = 0)
                Sum_squared += np.sum(np.power(feature, 2), axis = 0)
        
        # calculate the mean and the variance
        feat_mean = (Sum / total_frames) 
        feat_var = (Sum_squared / total_frames) - np.power(feat_mean, 2)
        feat_std = np.sqrt(feat_var)
        
        # save mean and variance information to out_file
        out_file = os.path.join(out_dir, 'mean_std.txt')
        print('Output file: %s' %(out_file))
        with open(out_file, mode = 'w') as file_o:
            file_o.write('mean\n')
            for i in range(np.size(feat_mean)):
                file_o.write('%e '% (feat_mean[i]))
            file_o.write('\n')
            for i in range(np.size(feat_var)):
                file_o.write('%e '% (feat_var[i]))
            file_o.write('\n')